In [2]:
import os
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin
import numpy as np

In [3]:
class CategoricalImputer(TransformerMixin):
    def fit(self, X, y=None):
        # uniques, counts = np.unique(X, return_counts=True)
        self.fill = pd.Series([X[c].value_counts().index[0]
                               if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
                              index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [12]:
data_dir = '/home/joan/PycharmProjects/titanic/data'
train_file = os.path.join(data_dir, 'train.csv')

In [ ]:
data = pd.read_csv(train_file)
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [26]:
data[['Age', 'Fare']] = Imputer().fit_transform(data[['Age', 'Fare']].values)

data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [36]:
embark_imputer = CategoricalImputer()
data['Embarked'] = embark_imputer.fit_transform(data[['Embarked']])

In [37]:
data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [53]:
data.groupby(['Sex'])['Survived'].sum() * 100 /  data.groupby(['Sex'])['Survived'].count()

Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64

In [50]:
data.groupby(['Sex']).count()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked
Sex,,,,,,,
female,314,314,314,314,314,314,314
male,577,577,577,577,577,577,577


In [54]:
data.groupby(['Pclass'])['Survived'].sum() * 100 / data.groupby(['Pclass'])['Survived'].count()

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64

In [56]:
attr = 'SibSp'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

SibSp
0    34.539474
1    53.588517
2    46.428571
3    25.000000
4    16.666667
5     0.000000
8     0.000000
Name: Survived, dtype: float64

In [57]:
attr = 'Parch'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

Parch
0    34.365782
1    55.084746
2    50.000000
3    60.000000
4     0.000000
5    20.000000
6     0.000000
Name: Survived, dtype: float64

In [58]:
attr = 'Embarked'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

Embarked
C    55.357143
Q    38.961039
S    33.900929
Name: Survived, dtype: float64

In [59]:
def cat_to_num(series):
    series = seris.astype('category')
    return series.cate.codes